In [2]:
import base64
import datetime
import os
import json
import pandas as pd
import time


import google.auth
import numpy as np
import tensorflow.io as tf_io

from concurrent import futures
from google.cloud import bigquery
from typing import List, Optional, Text, Tuple
from tqdm import tqdm

In [3]:
PROJECT_ID = 'jk-mlops-dev'
BUCKET_NAME = 'jk-ann-staging'
REGION = 'us-central1'
BQ_LOCATION = 'US'
BQ_DATASET_NAME = 'test'

In [4]:
credentials, _ = google.auth.default()
authed_session = google.auth.transport.requests.AuthorizedSession(credentials)

In [25]:
parent = 'https://us-central1-aiplatform.googleapis.com/v1alpha1/projects/895222332033/locations/us-central1'

In [26]:
api_url = f'{parent}/indexes'
response = authed_session.get(api_url)
response.json()

{'indexes': [{'name': 'projects/895222332033/locations/us-central1/indexes/1246371196874784768',
   'displayName': 'Song embeddings',
   'description': 'Song embeddings',
   'metadataSchemaUri': 'gs://google-cloud-aiplatform/schema/index/metadata/nearest_neighbor_search_1.0.0.yaml',
   'metadata': {'config': {'dimensions': 50,
     'approximateNeighborsCount': 50,
     'distanceMeasureType': 'DOT_PRODUCT_DISTANCE',
     'featureNormType': 'UNIT_L2_NORM',
     'treeAhConfig': {'childNodeCount': 1000, 'maxLeavesToSearch': 200}}},
   'etag': 'AMEw9yM9pHarA31vgrw_cYxQkpqibM--KNBiBlAj_4q0EZuKXiT_SrobYzhWa1AkKic=',
   'createTime': '2020-12-23T16:22:45.213115Z',
   'updateTime': '2020-12-23T16:36:24.214719Z'},
  {'name': 'projects/895222332033/locations/us-central1/indexes/1444529580479086592',
   'displayName': 'Song embeddings',
   'description': 'Song embeddings',
   'metadataSchemaUri': 'gs://google-cloud-aiplatform/schema/index/metadata/nearest_neighbor_search_1.0.0.yaml',
   'metadata'

In [18]:
response.reason

'OK'

In [51]:
api_url = f'{parent}/indexes/7209137103513321472'
response = authed_session.delete(api_url)
response.json()

{'name': 'projects/895222332033/locations/us-central1/operations/1396733810019663872',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeleteOperationMetadata',
  'genericMetadata': {'createTime': '2020-12-23T16:19:24.604334Z',
   'updateTime': '2020-12-23T16:19:24.604334Z'}},
 'done': True,
 'response': {'@type': 'type.googleapis.com/google.protobuf.Empty'}}

In [28]:
api_url = f'{parent}/indexEndpoints/5491013845671477248'

response = authed_session.get(api_url)
response.json()

{'name': 'projects/895222332033/locations/us-central1/indexEndpoints/5491013845671477248',
 'displayName': 'Endpoint for Song embeddings',
 'etag': 'AMEw9yN44_RygUF8dGV-BQU54h1EUxFHCGIT7putId8SxwoXc0wittIxgLYGqMx3hco=',
 'createTime': '2020-12-22T05:19:16.911316Z',
 'updateTime': '2020-12-22T05:19:34.076591Z',
 'network': 'projects/895222332033/global/networks/default'}

In [37]:
api_url = f'{parent}/operations/4968545711361622016:cancel'

response = authed_session.post(api_url)
response.json()

{'error': {'code': 400,
  'message': 'Operation projects/895222332033/locations/us-central1/operations/4968545711361622016 is not cancellable.',
  'status': 'FAILED_PRECONDITION'}}

In [40]:
api_url = f'{parent}/operations/4968545711361622016'

response = authed_session.get(api_url)
response.json()

{'name': 'projects/895222332033/locations/us-central1/indexEndpoints/5491013845671477248/operations/4968545711361622016',
 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeployIndexOperationMetadata',
  'genericMetadata': {'createTime': '2020-12-22T05:19:37.729306Z',
   'updateTime': '2020-12-22T09:21:01.683905Z'}},
 'done': True,
 'error': {'code': 13, 'message': 'INTERNAL'}}

In [41]:
api_url = f'{parent}/operations'

response = authed_session.get(api_url)
response.json()

{'operations': [{'name': 'projects/895222332033/locations/us-central1/operations/6473873886810210304',
   'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeleteOperationMetadata',
    'genericMetadata': {'createTime': '2020-12-22T14:25:52.525039Z',
     'updateTime': '2020-12-22T14:25:52.525039Z'}},
   'done': True,
   'response': {'@type': 'type.googleapis.com/google.protobuf.Empty'}},
  {'name': 'projects/895222332033/locations/us-central1/indexEndpoints/5491013845671477248/operations/4968545711361622016',
   'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeployIndexOperationMetadata',
    'genericMetadata': {'createTime': '2020-12-22T05:19:37.729306Z',
     'updateTime': '2020-12-22T09:21:01.683905Z'}},
   'done': True,
   'error': {'code': 13, 'message': 'INTERNAL'}},
  {'name': 'projects/895222332033/locations/us-central1/operations/48363118459355136',
   'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1a

In [11]:
api_url = f'{parent}/operations'

response = authed_session.get(api_url)
response.json()

{'operations': [{'name': 'projects/895222332033/locations/us-central1/indexEndpoints/5491013845671477248/operations/4968545711361622016',
   'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeployIndexOperationMetadata',
    'genericMetadata': {'createTime': '2020-12-22T05:19:37.729306Z',
     'updateTime': '2020-12-22T05:19:37.729306Z'}}},
  {'name': 'projects/895222332033/locations/us-central1/indexEndpoints/5491013845671477248/operations/4840756071934984192',
   'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.CreateIndexEndpointOperationMetadata',
    'genericMetadata': {'createTime': '2020-12-22T05:19:16.911316Z',
     'updateTime': '2020-12-22T05:19:34.084783Z'}},
   'done': True,
   'response': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.IndexEndpoint',
    'name': 'projects/895222332033/locations/us-central1/indexEndpoints/5491013845671477248'}},
  {'name': 'projects/895222332033/locations/us-central1/ind

In [3]:
def wait_for_result(query_job):
    print("Executing query with job ID: {}".format(query_job.job_id))
    start_time = time.time()
    while True:
        print("\rQuery executing: {:0.2f}s".format(time.time() - start_time), end="")
        try:
            query_job.result(timeout=0.5)
            break
        except futures.TimeoutError:
            continue
    print("\nQuery complete after {:0.2f}s".format(time.time() - start_time))
    return query_job

In [ ]:
number_of_records = 3000000

query = f"""
    CREATE OR REPLACE TABLE `{BQ_DATASET_NAME}.playlist`
    AS
    SELECT DISTINCT
        id list_Id, 
        tracks_data_id track_Id, 
        tracks_data_title track_title,
        tracks_data_artist_name track_artist
    FROM `bigquery-samples.playlists.playlist`
    WHERE tracks_data_title IS NOT NULL AND tracks_data_id > 0
"""

if  isinstance(number_of_records, int) and number_of_records <= 12304975 :
    limit_clause = f'LIMIT {number_of_records}'
    query = query + limit_clause
    
query_job = client.query(query)
query_job = wait_for_result(query_job)

In [4]:
query_template = """

CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.playlist
(
    list_id INT64,
    track_id INT64,
    track_title STRING,
    track_artist STRING
);

CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.item_cooc
(
    item1_Id INT64,
    item2_Id INT64,
    cooc INT64,
    pmi FLOAT64
);

CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.valid_item_groups
(
    item_Id INT64,
    group_Id INT64
);


CREATE OR REPLACE VIEW `{BQ_DATASET_NAME}.vw_item_groups`
    AS
    SELECT
      list_Id AS group_Id,
      track_Id AS item_Id
    FROM  
      `{BQ_DATASET_NAME}.playlist`; 
      
CREATE OR REPLACE MODEL {BQ_DATASET_NAME}.item_matching_model
    OPTIONS(
        MODEL_TYPE='matrix_factorization', 
        FEEDBACK_TYPE='implicit',
        WALS_ALPHA=1,
        NUM_FACTORS=10,
        USER_COL='item1_Id', 
        ITEM_COL='item2_Id',
        RATING_COL='score',
        DATA_SPLIT_METHOD='no_split'
    )
    AS
    SELECT 
        item1_Id, 
        item2_Id, 
        cooc * pmi AS score
    FROM {BQ_DATASET_NAME}.item_cooc;
      
    
CREATE OR REPLACE PROCEDURE {BQ_DATASET_NAME}.sp_ComputePMI(
    IN min_item_frequency INT64,
    IN max_group_size INT64
    )

    BEGIN

    DECLARE total INT64;

    # Get items with minimum frequency
    CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.valid_item_groups
    AS

    # Create valid item set
    WITH 
    valid_items AS (
        SELECT item_Id, COUNT(group_Id) AS item_frequency
        FROM {BQ_DATASET_NAME}.vw_item_groups
        GROUP BY item_Id
        HAVING item_frequency >= min_item_frequency
    ),

    # Create valid group set
    valid_groups AS (
        SELECT group_Id, COUNT(item_Id) AS group_size
        FROM {BQ_DATASET_NAME}.vw_item_groups
        WHERE item_Id IN (SELECT item_Id FROM valid_items)
        GROUP BY group_Id
        HAVING group_size BETWEEN 2 AND max_group_size
    )

    SELECT item_Id, group_Id
    FROM {BQ_DATASET_NAME}.vw_item_groups
    WHERE item_Id IN (SELECT item_Id FROM valid_items)
    AND group_Id IN (SELECT group_Id FROM valid_groups);

    # Compute pairwise cooc
    CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.item_cooc
    AS
    SELECT item1_Id, item2_Id, SUM(cooc) AS cooc
    FROM
    (
        SELECT
        a.item_Id item1_Id,
        b.item_Id item2_Id,
        1 as cooc
        FROM {BQ_DATASET_NAME}.valid_item_groups a
        JOIN {BQ_DATASET_NAME}.valid_item_groups b
        ON a.group_Id = b.group_Id
        AND a.item_Id < b.item_Id
    )
    GROUP BY  item1_Id, item2_Id;

    ###################################
    
    # Compute item frequencies
    CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.item_frequency
    AS
    SELECT item_Id, COUNT(group_Id) AS frequency
    FROM {BQ_DATASET_NAME}.valid_item_groups
    GROUP BY item_Id;

    ###################################
    
    # Compute total frequency |D|
    SET total = (
        SELECT SUM(frequency)  AS total
        FROM {BQ_DATASET_NAME}.item_frequency
    );
    
    ###################################
    
    # Add mirror item-pair cooc and same item frequency as cooc
    CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.item_cooc
    AS
    SELECT item1_Id, item2_Id, cooc
    FROM {BQ_DATASET_NAME}.item_cooc
    UNION ALL
    SELECT item2_Id as item1_Id, item1_Id AS item2_Id, cooc
    FROM {BQ_DATASET_NAME}.item_cooc
    UNION ALL
    SELECT item_Id as item1_Id, item_Id AS item2_Id, frequency as cooc
    FROM {BQ_DATASET_NAME}.item_frequency;

    ###################################
    
    # Compute PMI
    CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.item_cooc
    AS
    SELECT
        a.item1_Id,
        a.item2_Id,
        a.cooc,
        LOG(a.cooc, 2) - LOG(b.frequency, 2) - LOG(c.frequency, 2) + LOG(total, 2) AS pmi
    FROM {BQ_DATASET_NAME}.item_cooc a
    JOIN {BQ_DATASET_NAME}.item_frequency b
    ON a.item1_Id = b.item_Id
    JOIN {BQ_DATASET_NAME}.item_frequency c
    ON a.item2_Id = c.item_Id; 
    END;
    
CREATE OR REPLACE PROCEDURE {BQ_DATASET_NAME}.sp_TrainItemMatchingModel(
        IN dimensions INT64
    )

    BEGIN

    CREATE OR REPLACE MODEL {BQ_DATASET_NAME}.item_matching_model
    OPTIONS(
        MODEL_TYPE='matrix_factorization', 
        FEEDBACK_TYPE='implicit',
        WALS_ALPHA=1,
        NUM_FACTORS=(dimensions),
        USER_COL='item1_Id', 
        ITEM_COL='item2_Id',
        RATING_COL='score',
        DATA_SPLIT_METHOD='no_split'
    )
    AS
    SELECT 
        item1_Id, 
        item2_Id, 
        cooc * pmi AS score
    FROM {BQ_DATASET_NAME}.item_cooc;
    END;
    
CREATE OR REPLACE PROCEDURE {BQ_DATASET_NAME}.sp_ExractEmbeddings() 
    BEGIN
    CREATE OR REPLACE TABLE {BQ_DATASET_NAME}.item_embeddings AS
        WITH 
        step1 AS
        (
            SELECT 
                feature AS item_Id,
                factor_weights
            FROM
                ML.WEIGHTS(MODEL `{BQ_DATASET_NAME}..item_matching_model`)
            WHERE feature != 'global__INTERCEPT__'
        ),

        step2 AS
        (
            SELECT 
                item_Id, 
                factor, 
                SUM(weight) AS weight
            FROM step1,
            UNNEST(step1.factor_weights) AS embedding
            GROUP BY 
            item_Id,
            factor 
        )

        SELECT 
            item_Id as id, 
            ARRAY_AGG(weight ORDER BY factor ASC) embedding,
        FROM step2
        GROUP BY item_Id;
    END 
"""

In [5]:
query = query_template.format(BQ_DATASET_NAME=BQ_DATASET_NAME)
print(query)



CREATE OR REPLACE TABLE test.playlist
(
    list_id INT64,
    track_id INT64,
    track_title STRING,
    track_artist STRING
);

CREATE OR REPLACE TABLE test.item_cooc
(
    item1_Id INT64,
    item2_Id INT64,
    cooc INT64,
    pmi FLOAT64
);

CREATE OR REPLACE TABLE test.valid_item_groups
(
    item_Id INT64,
    group_Id INT64
);


CREATE OR REPLACE VIEW `test.vw_item_groups`
    AS
    SELECT
      list_Id AS group_Id,
      track_Id AS item_Id
    FROM  
      `test.playlist`; 
      
CREATE OR REPLACE MODEL test.item_matching_model
    OPTIONS(
        MODEL_TYPE='matrix_factorization', 
        FEEDBACK_TYPE='implicit',
        WALS_ALPHA=1,
        NUM_FACTORS=10,
        USER_COL='item1_Id', 
        ITEM_COL='item2_Id',
        RATING_COL='score',
        DATA_SPLIT_METHOD='no_split'
    )
    AS
    SELECT 
        item1_Id, 
        item2_Id, 
        cooc * pmi AS score
    FROM test.item_cooc;
      
    
CREATE OR REPLACE PROCEDURE test.sp_ComputePMI(
    IN min_ite

In [7]:
client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

In [8]:
query_job = client.query(query)
query_job.result()

BadRequest: 400 Query error: Input data doesn't contain any rows. at [34:1]

(job ID: cdf0cf04-e9ec-48b0-a26d-f2a626ff4b1b)

                               -----Query Job SQL Follows-----                                

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:
   3:CREATE OR REPLACE TABLE test.playlist
   4:(
   5:    list_id INT64,
   6:    track_id INT64,
   7:    track_title STRING,
   8:    track_artist STRING
   9:);
  10:
  11:CREATE OR REPLACE TABLE test.item_cooc
  12:(
  13:    item1_Id INT64,
  14:    item2_Id INT64,
  15:    cooc INT64,
  16:    pmi FLOAT64
  17:);
  18:
  19:CREATE OR REPLACE TABLE test.valid_item_groups
  20:(
  21:    item_Id INT64,
  22:    group_Id INT64
  23:);
  24:
  25:
  26:CREATE OR REPLACE VIEW `test.vw_item_groups`
  27:    AS
  28:    SELECT
  29:      list_Id AS group_Id,
  30:      track_Id AS item_Id
  31:    FROM  
  32:      `test.playlist`; 
  33:      
  34:CREATE OR REPLACE MODEL